In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()

sc.stop()
conf = pyspark.SparkConf()
conf.set('spark.local.dir', '/Volumes/My Passport')
sc = pyspark.SparkContext(conf=conf)


In [2]:
sqlContext = SQLContext (sc)
sc


<SparkContext master=local[*] appName=PySparkShell>

In [ ]:
## Cleaning Positive and Negative Data Records and Merging them

In [37]:
spark.conf.set('spark.sql.shuffle.partitions', 4)
csv_file_path1 = '/Volumes/My Passport/OpenCorpus/data-neg.csv/neg.csv'
#csv_file_path2 = '/Volumes/My Passport/OpenCorpus/data-pos.csv/2.csv'
df = sqlContext.read.csv(csv_file_path1 )
df.printSchema()


root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



In [38]:
df.head()

Row(_c0='0d0467c4cccb0dcf0a30589a1d3dddd2cecfda11', _c1='ce994a42bee6f083f27071840eb79ab605f5760d', _c2="Falsified and substandard drugs are a global health problem, particularly in low- and middle-income countries (LMIC) that have weak pharmacovigilance and drug regulatory systems. Poor quality medicines have important health consequences, including the potential for treatment failure, development of antimicrobial resistance, and serious adverse drug reactions, increasing healthcare costs and undermining the public's confidence in healthcare systems. This article presents a review of the methods employed for the analysis of pharmaceutical formulations. Technologies for detecting substandard and falsified drugs were identified primarily through literature reviews. Key-informant interviews with experts augmented our methods when warranted. In order to aid comparisons, technologies were assigned a suitability score for use in LMIC ranging from 0–8. Scores measured the need for electricit

In [39]:
df.count()

58675

In [40]:
from pyspark.sql.functions import udf, concat, col, lit,regexp_replace, when, isnan, trim
df1 = df.na.drop(subset=["_c0" , "_c1", "_c2", "_c3", "_c4"])
def to_null(c):
    return when(~(col(c).isNull() | isnan(col(c)) | (trim(col(c)) == "")), col(c))


df11 = df1.select([to_null(c).alias(c) for c in df.columns]).na.drop()

In [41]:
df11.show()

+--------------------+--------------------+--------------------+--------------------+---+
|                 _c0|                 _c1|                 _c2|                 _c3|_c4|
+--------------------+--------------------+--------------------+--------------------+---+
|0d0467c4cccb0dcf0...|ce994a42bee6f083f...|Falsified and sub...|Abstract - This r...|  0|
|878a852598fbb063b...|a7dbf9168b16ca1de...|Objectives To exp...|With the introduc...|  0|
|237e5a3778ccff06c...|8ebd5cf0eaf77f077...|Abstract The Ni 0...|A significantly i...|  0|
|878a852598fbb063b...|a77eb24170a610ead...|Objectives To exp...|Abstract Games-ba...|  0|
|771723e7e2d9ed97d...|d6a8f6cc558225fb2...|In the future, th...|Abstract Experime...|  0|
|237e5a3778ccff06c...|08dc950a0600d81c6...|Abstract The Ni 0...|Sex differences i...|  0|
|0d0467c4cccb0dcf0...|6cc77df0f67980486...|Falsified and sub...|African American-...|  0|
|771723e7e2d9ed97d...|ef8cd11edb3ad918e...|In the future, th...|This paper presen...|  0|
|062e9c757

In [42]:
df2 = df11.filter(df11._c4.isin(['0','1']))

In [43]:
df2.show()

+--------------------+--------------------+--------------------+--------------------+---+
|                 _c0|                 _c1|                 _c2|                 _c3|_c4|
+--------------------+--------------------+--------------------+--------------------+---+
|0d0467c4cccb0dcf0...|ce994a42bee6f083f...|Falsified and sub...|Abstract - This r...|  0|
|878a852598fbb063b...|a7dbf9168b16ca1de...|Objectives To exp...|With the introduc...|  0|
|237e5a3778ccff06c...|8ebd5cf0eaf77f077...|Abstract The Ni 0...|A significantly i...|  0|
|878a852598fbb063b...|a77eb24170a610ead...|Objectives To exp...|Abstract Games-ba...|  0|
|771723e7e2d9ed97d...|d6a8f6cc558225fb2...|In the future, th...|Abstract Experime...|  0|
|237e5a3778ccff06c...|08dc950a0600d81c6...|Abstract The Ni 0...|Sex differences i...|  0|
|0d0467c4cccb0dcf0...|6cc77df0f67980486...|Falsified and sub...|African American-...|  0|
|771723e7e2d9ed97d...|ef8cd11edb3ad918e...|In the future, th...|This paper presen...|  0|
|062e9c757

In [44]:
import re
from pyspark.sql.functions import udf, concat, col, lit,regexp_replace
from pyspark.sql.types import FloatType
#commaRep = udf(lambda x: re.sub(',$|^,','', x))
#commaToDot = udf(lambda x : float(str(x).replace(',', ' ')), FloatType())

In [45]:
dfNew = df.withColumn("abs11", regexp_replace(col("_c2"), "[,]", "")).withColumn("abs2", regexp_replace(col("_c3"), "[,]", ""))

In [46]:
dfNew.show()

+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+
|                 _c0|                 _c1|                 _c2|                 _c3|_c4|               abs11|                abs2|
+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+
|0d0467c4cccb0dcf0...|ce994a42bee6f083f...|Falsified and sub...|Abstract - This r...|  0|Falsified and sub...|Abstract - This r...|
|878a852598fbb063b...|a7dbf9168b16ca1de...|Objectives To exp...|With the introduc...|  0|Objectives To exp...|With the introduc...|
|237e5a3778ccff06c...|8ebd5cf0eaf77f077...|Abstract The Ni 0...|A significantly i...|  0|Abstract The Ni 0...|A significantly i...|
|878a852598fbb063b...|a77eb24170a610ead...|Objectives To exp...|Abstract Games-ba...|  0|Objectives To exp...|Abstract Games-ba...|
|771723e7e2d9ed97d...|d6a8f6cc558225fb2...|In the future, th...|Abstract Exp

In [47]:
dfNew.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- abs11: string (nullable = true)
 |-- abs2: string (nullable = true)



In [48]:
dfNew_T = dfNew.withColumn("t1", regexp_replace(col("_c0"), "[,]", "")).withColumn("t2", regexp_replace(col("_c1"), "[,]", ""))

In [49]:
dfNew_T.show()

+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+--------------------+--------------------+
|                 _c0|                 _c1|                 _c2|                 _c3|_c4|               abs11|                abs2|                  t1|                  t2|
+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+--------------------+--------------------+
|0d0467c4cccb0dcf0...|ce994a42bee6f083f...|Falsified and sub...|Abstract - This r...|  0|Falsified and sub...|Abstract - This r...|0d0467c4cccb0dcf0...|ce994a42bee6f083f...|
|878a852598fbb063b...|a7dbf9168b16ca1de...|Objectives To exp...|With the introduc...|  0|Objectives To exp...|With the introduc...|878a852598fbb063b...|a7dbf9168b16ca1de...|
|237e5a3778ccff06c...|8ebd5cf0eaf77f077...|Abstract The Ni 0...|A significantly i...|  0|Abstract The Ni 0...|A significantly i...

In [50]:
dfNew_T = dfNew_T.na.drop()
dfNew_T.show()

+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+--------------------+--------------------+
|                 _c0|                 _c1|                 _c2|                 _c3|_c4|               abs11|                abs2|                  t1|                  t2|
+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+--------------------+--------------------+
|0d0467c4cccb0dcf0...|ce994a42bee6f083f...|Falsified and sub...|Abstract - This r...|  0|Falsified and sub...|Abstract - This r...|0d0467c4cccb0dcf0...|ce994a42bee6f083f...|
|878a852598fbb063b...|a7dbf9168b16ca1de...|Objectives To exp...|With the introduc...|  0|Objectives To exp...|With the introduc...|878a852598fbb063b...|a7dbf9168b16ca1de...|
|237e5a3778ccff06c...|8ebd5cf0eaf77f077...|Abstract The Ni 0...|A significantly i...|  0|Abstract The Ni 0...|A significantly i...

In [51]:
import pyspark.sql.functions as size
dfNew_T = dfNew_T.withColumn('t3', when(col('_c4') == '', 0).otherwise(col('_c4')))

In [52]:
dfNew_T.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- abs11: string (nullable = true)
 |-- abs2: string (nullable = true)
 |-- t1: string (nullable = true)
 |-- t2: string (nullable = true)
 |-- t3: string (nullable = true)



In [53]:
dfNew_1 = dfNew_T.select(col("t1").alias('id1'), col('t2').alias('id2'),col('abs11').alias('paperAbstract1'),col('abs2').alias('paperAbstract2'), col('_c4').alias('label'))
dfNew_1.head()

Row(id1='0d0467c4cccb0dcf0a30589a1d3dddd2cecfda11', id2='ce994a42bee6f083f27071840eb79ab605f5760d', paperAbstract1="Falsified and substandard drugs are a global health problem particularly in low- and middle-income countries (LMIC) that have weak pharmacovigilance and drug regulatory systems. Poor quality medicines have important health consequences including the potential for treatment failure development of antimicrobial resistance and serious adverse drug reactions increasing healthcare costs and undermining the public's confidence in healthcare systems. This article presents a review of the methods employed for the analysis of pharmaceutical formulations. Technologies for detecting substandard and falsified drugs were identified primarily through literature reviews. Key-informant interviews with experts augmented our methods when warranted. In order to aid comparisons technologies were assigned a suitability score for use in LMIC ranging from 0–8. Scores measured the need for elect

In [54]:
dfNew_2 = dfNew_1.filter(dfNew_1.label.isin(['0','1']))

In [55]:
from pyspark.sql.functions import udf, concat, col, lit,regexp_replace, when, isnan, trim
def to_null(c):
    return when(~(col(c).isNull() | isnan(col(c)) | (trim(col(c)) == "")), col(c))


dfNew_21 = dfNew_2.select([to_null(c).alias(c) for c in dfNew_2.columns]).na.drop()

In [56]:
df13 = dfNew_21.na.drop(subset=["id1" , "id2", "paperAbstract1", "paperAbstract2", "label"])
df21 = df13.dropDuplicates()

In [57]:
df21.count()

40514

In [58]:
df21.show(10)

+--------------------+--------------------+--------------------+--------------------+-----+
|                 id1|                 id2|      paperAbstract1|      paperAbstract2|label|
+--------------------+--------------------+--------------------+--------------------+-----+
|878a852598fbb063b...|f2fec74b3fcf04509...|Objectives To exp...|Lewis et al aimed...|    0|
|771723e7e2d9ed97d...|ecda4d12ebbaaecc0...|In the future the...|Abstract The chem...|    0|
|878a852598fbb063b...|a77eb24170a610ead...|Objectives To exp...|Recent studies ha...|    0|
|984774e366d3d4fcf...|5f8bc6319446f38d6...|BackgroundCorneal...|BackgroundSalicyl...|    0|
|771723e7e2d9ed97d...|f2fa93e36b2eb4334...|In the future the...|In this letter we...|    0|
|984774e366d3d4fcf...|44c481f3f17a8b1e9...|BackgroundCorneal...|The quandary in t...|    0|
|771723e7e2d9ed97d...|e3718ee0bafe60b68...|In the future the...|The purpose of th...|    0|
|984774e366d3d4fcf...|34f25a8704614163c...|BackgroundCorneal...|Abattoir waste m

In [59]:
data = df21
data.repartition(1).write.option("quoteMode", "None").option("header","true").csv("/Volumes/My Passport/OpenCorpus/OC-Data/neg-samples.csv")

In [ ]:
## Reused the above code for cleaning both the positive and negative samples already generated -> 
## (Changed the input file names and output file name)
## After that, ran the code below for merging the two files

In [ ]:
## =================== Merging Code ===================== #####

In [63]:
### Merging the data

spark.conf.set('spark.sql.shuffle.partitions', 4)
csv_file_path_1 = 'Desktop/open-pos.csv'
csv_file_path_2 = 'Desktop/open-neg.csv'
csv_file_path = 'Desktop/open-corpus.csv'

dataframe_1 = sqlContext.read.option('header', 'true').option("inferSchema","true").csv([csv_file_path_1, csv_file_path_2])
dataframe_1.printSchema()
dataframe_1.count()

root
 |-- id1: string (nullable = true)
 |-- id2: string (nullable = true)
 |-- paperAbstract1: string (nullable = true)
 |-- paperAbstract2: string (nullable = true)
 |-- label: integer (nullable = true)



'\ndataframe_11 = dataframe_1.limit(60000)\n\ndataframe_2 = sqlContext.read.option(\'header\', \'true\').option("inferSchema","true").csv(csv_file_path_2)\ndataframe_2.printSchema()\ndataframe_2.count()\n\ndataframe_22 = dataframe_2.limit(40000)\n'

In [ ]:

dataframe_1.count()

In [65]:
dataframe_1.show()

+--------------------+--------------------+--------------------+--------------------+-----+
|                 id1|                 id2|      paperAbstract1|      paperAbstract2|label|
+--------------------+--------------------+--------------------+--------------------+-----+
|b879c5dfe1d691d44...|ec582ace310706d01...|Cell rearrangemen...|Cadherins not onl...|    1|
|7a76880fa360569d9...|0a971c292d32b82f1...|Background Nutrit...|The incidence of ...|    1|
|060e29dab03e478e3...|77205fc81b913939c...|The objective of ...|An effective host...|    1|
|309c4700c350e3f7d...|c1d3a76a61e17e7b0...|Pulsed field grad...|Abstract Crystall...|    1|
|c2b9906375d2582fc...|4e36ed3f4f8fda2ab...|Nanofibrous bioma...|Abstract Gold nan...|    1|
|b3e4a519c09e2b2df...|1c761e56b492a1a02...|The purpose of th...|Organizational le...|    1|
|54b94c389de1a1381...|5f0aa2614821d4f2a...|Breast cancer is ...|Chemical investig...|    1|
|6ac8999bc3e4c8843...|a94b5adc6be4b10d1...|The privacy by de...|Online social ne

In [62]:
csv_file_path = 'Desktop/open-corpus-1.csv'
dataframe_1.coalesce(1).write.format('csv').option('header', 'true').save(csv_file_path)